<a href="https://colab.research.google.com/github/ParitKansal/hugging-face-nlp/blob/main/Hugging_Face_NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate

In [ ]:
from datasets import load_dataset  # Importing the `load_dataset` function from the Hugging Face Datasets library
from transformers import AutoTokenizer, DataCollatorWithPadding  # Importing tools for tokenization and data collation

# Load the GLUE MRPC dataset
raw_datasets = load_dataset("glue", "mrpc")  # The GLUE MRPC dataset contains sentence pairs for paraphrase detection.

# Define the checkpoint for the tokenizer
checkpoint = "bert-base-uncased"  # Use the BERT base model with uncased tokenization
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # Load the tokenizer associated with the specified checkpoint

# Define a function to tokenize input examples
def tokenize_function(example):
    # Tokenize the sentence pairs with truncation to fit the model's input size
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Apply the tokenize function to the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Use the map method with `batched=True` to tokenize all examples efficiently in batches

# Create a data collator for dynamically padding input data
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# This ensures input tensors are padded to the longest sequence in a batch, making them ready for model training

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
# Remove unnecessary columns from the tokenized dataset
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# The columns "sentence1", "sentence2", and "idx" are no longer needed after tokenization,
# so we remove them to keep only the relevant data for model training.

# Rename the "label" column to "labels"
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# The model expects the column containing target values to be named "labels",
# so we rename the "label" column accordingly.

# Set the dataset format to PyTorch tensors
tokenized_datasets.set_format("torch")
# This ensures the dataset outputs are in a format compatible with PyTorch during training.

# Display the column names of the training dataset
tokenized_datasets["train"].column_names
# This lists the final column names in the training dataset to verify the structure.

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader  # Import DataLoader to handle batching and shuffling of data

# Create a DataLoader for the training dataset
train_dataloader = DataLoader(
    tokenized_datasets["train"],  # Use the training split of the tokenized dataset
    shuffle=True,  # Shuffle the data during training to improve model generalization
    batch_size=8,  # Process data in batches of 8 samples
    collate_fn=data_collator  # Use the data collator for dynamic padding of input sequences
)

# Create a DataLoader for the validation dataset
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],  # Use the validation split of the tokenized dataset
    batch_size=8,  # Process data in batches of 8 samples
    collate_fn=data_collator  # Use the same data collator for consistency
)

# The DataLoaders handle the tokenized datasets and ensure the data is prepared for the model
# in the correct format with batching, padding, and shuffling (for training).

In [ ]:
# Iterate over the training DataLoader and retrieve the first batch
for batch in train_dataloader:
    break  # Break after retrieving the first batch

# Display the shape of each element in the batch
{k: v.shape for k, v in batch.items()}
# Create a dictionary comprehension to display the shapes of all elements (tensors) in the batch.
# Keys (`k`) represent the data types (e.g., 'input_ids', 'attention_mask', 'labels'),
# and values (`v.shape`) represent the dimensions of the corresponding tensors.

# This provides a quick summary of the batch structure, ensuring that the input format is as expected for model training.

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 69]),
 'token_type_ids': torch.Size([8, 69]),
 'attention_mask': torch.Size([8, 69])}

In [ ]:
from transformers import AutoModelForSequenceClassification  # Import the model class for sequence classification

# Load a pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,  # Use the same checkpoint as the tokenizer (e.g., "bert-base-uncased")
    num_labels=2  # Specify the number of labels for the classification task (binary classification in this case)
)

# The model is initialized with the pre-trained weights from the specified checkpoint,
# and the classification head is adjusted to output predictions for two classes.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Pass the batch through the model to get outputs
outputs = model(**batch)
# The double-asterisk (`**`) unpacks the batch dictionary, passing its items as keyword arguments to the model.
# Typical inputs include 'input_ids', 'attention_mask', and 'labels' for training.

# Print the loss and logits from the model's outputs
print(outputs.loss, outputs.logits.shape)
# `outputs.loss` contains the computed loss for the batch (only if 'labels' are provided in the input).
# `outputs.logits` contains the raw predictions (logits) from the model, which have the shape [batch_size, num_labels].
# This helps verify that the model is processing the batch correctly and outputting the expected results.

tensor(0.7830, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [ ]:
from transformers import AdamW  # Import the AdamW optimizer from the Hugging Face transformers library

# Initialize the AdamW optimizer for training
optimizer = AdamW(
    model.parameters(),  # Pass the model's parameters to the optimizer, so it knows which parameters to update
    lr=5e-5  # Set the learning rate to 5e-5 (a commonly used learning rate for fine-tuning transformers)
)

# AdamW (Adam with Weight Decay) is an adaptive learning rate optimization algorithm,
# often used for training transformer-based models like BERT. The learning rate controls how much
# to adjust the model's parameters with respect to the computed gradients during training.

In [ ]:
from transformers import get_scheduler  # Import the function to get a learning rate scheduler

# Set the number of training epochs
num_epochs = 3  # The number of times the model will iterate over the entire training dataset

# Calculate the total number of training steps based on the number of epochs and the number of batches per epoch
num_training_steps = num_epochs * len(train_dataloader)
# Each epoch consists of several steps, one for each batch in the training dataloader.
# Multiply the number of epochs by the number of batches to get the total number of steps.

# Initialize the learning rate scheduler to adjust the learning rate during training
lr_scheduler = get_scheduler(
    "linear",  # Use a linear learning rate scheduler, where the learning rate decreases linearly from the initial value
    optimizer=optimizer,  # The optimizer to which the scheduler is applied
    num_warmup_steps=0,  # No warm-up steps, meaning the learning rate starts at the initial value immediately
    num_training_steps=num_training_steps,  # The total number of training steps
)

# Print the total number of training steps to verify the calculation
print(num_training_steps)

1377


In [ ]:
import torch  # Import the PyTorch library

# Check if a GPU (CUDA-enabled device) is available, otherwise use the CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# If CUDA is available, the model will use the GPU for faster training. Otherwise, it will default to the CPU.

# Move the model to the selected device (GPU or CPU)
model.to(device)
# This ensures that the model's parameters and computations are performed on the chosen device (GPU or CPU).

# Print the device (either 'cuda' or 'cpu') to verify which device is being used
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm  # Import the tqdm library for progress bars in the console

# Initialize the progress bar to track the number of training steps
progress_bar = tqdm(range(num_training_steps))  # num_training_steps was calculated earlier

# Set the model to training mode
model.train()

# Loop over the number of epochs
for epoch in range(num_epochs):
    # Iterate over the batches in the training DataLoader
    for batch in train_dataloader:
        # Move the batch to the correct device (GPU or CPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Pass the batch through the model
        outputs = model(**batch)
        # Get the loss from the model's outputs
        loss = outputs.loss

        # Backpropagate the loss to compute gradients
        loss.backward()

        # Update the model parameters using the optimizer
        optimizer.step()

        # Update the learning rate using the scheduler
        lr_scheduler.step()

        # Reset the gradients after the optimization step
        optimizer.zero_grad()

        # Update the progress bar by one step
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:
import evaluate  # Import the 'evaluate' library to handle evaluation metrics

# Load the GLUE MRPC evaluation metric
metric = evaluate.load("glue", "mrpc")

# Set the model to evaluation mode
model.eval()
# The model is switched to evaluation mode, which deactivates layers like dropout for inference.

# Iterate over the batches in the evaluation DataLoader
for batch in eval_dataloader:
    # Move the batch to the correct device (GPU or CPU)
    batch = {k: v.to(device) for k, v in batch.items()}

    # Disable gradient computation (no backpropagation during evaluation)
    with torch.no_grad():
        # Pass the batch through the model to get the outputs
        outputs = model(**batch)

    # Extract the logits (raw predictions before softmax)
    logits = outputs.logits

    # Get the predicted class by selecting the class with the highest logit value
    predictions = torch.argmax(logits, dim=-1)

    # Add the predictions and the true labels to the metric for evaluation
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Compute and return the final evaluation metrics (e.g., accuracy, F1-score)
metric.compute()

{'accuracy': 0.8602941176470589, 'f1': 0.902229845626072}

In [ ]:
import os

save_path = "/content/drive/My Drive/saved_model1"
os.makedirs(save_path, exist_ok=True)

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/saved_model1/tokenizer_config.json',
 '/content/drive/My Drive/saved_model1/special_tokens_map.json',
 '/content/drive/My Drive/saved_model1/vocab.txt',
 '/content/drive/My Drive/saved_model1/added_tokens.json',
 '/content/drive/My Drive/saved_model1/tokenizer.json')

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

# Optionally, save the optimizer and scheduler if you want to resume training later
torch.save(optimizer.state_dict(), "./saved_model/optimizer.pt")
torch.save(lr_scheduler.state_dict(), "./saved_model/lr_scheduler.pt")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

# If you want to load optimizer and scheduler for resuming training
optimizer.load_state_dict(torch.load("./saved_model/optimizer.pt"))
lr_scheduler.load_state_dict(torch.load("./saved_model/lr_scheduler.pt"))

<ipython-input-53-f456769acdae>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  optimizer.load_state_dict(torch.load("./saved_model/optimizer.pt"))
<ipython-input-53-f45676

In [ ]:
def make_prediction(text1, text2):
    # Tokenize the input text using the tokenizer
    inputs = tokenizer(text1, text2, truncation=True, padding=True, return_tensors="pt")

    # Move both inputs and the model to the same device
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the device
    model.to(device)  # Ensure the model is on the same device as inputs

    # Put the model in evaluation mode
    model.eval()

    # Perform inference without computing gradients
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the logits (raw predictions)
    logits = outputs.logits

    # Get the predicted class (0 or 1 for MRPC task)
    prediction = torch.argmax(logits, dim=-1).item()

    return prediction

# Example usage
text1 = "The quick brown fox jumps over the lazy dog."
text2 = "A fast brown fox leaps over a lazy dog."
prediction = make_prediction(text1, text2)

print(f"Prediction: {prediction}")  # 0 or 1 based on the MRPC task (paraphrase or not)

Prediction: 1
